In [1]:
import os
import glob 
import numpy as np
import pandas as pd 
import scanpy as sc
import seaborn as sns

Collect Cellranger outputs for both datasets

## Kinker et al

In [4]:
kinker_metadata = pd.read_csv('/node200data/18parkky/datasets/data/public/processed_data/Kinker_et_al/Metadata.txt', sep='\t')
adata = sc.read_h5ad('/node200data/18parkky/datasets/data/public/processed_data/Kinker_et_al/CPM_data.h5ad')

/tmp/ipykernel_155434/1346728635.py:1: DtypeWarning: Columns (4,9,10,11,12,13,14,15,16,17,18,19,20) have mixed types. Specify dtype option on import or set low_memory=False.
  kinker_metadata = pd.read_csv('/node200data/18parkky/datasets/data/public/processed_data/Kinker_et_al/Metadata.txt', sep='\t')
/home/18parkky/anaconda3-2021.11/envs/sc/lib/python3.12/site-packages/anndata/_core/aligned_df.py:67: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)
/home/18parkky/anaconda3-2021.11/envs/sc/lib/python3.12/site-packages/anndata/_core/aligned_df.py:67: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)


In [23]:
dict_CB_to_MetaData = { tup.NAME : [tup.Cell_line, tup.Pool_ID, tup.Cancer_type] for tup in kinker_metadata.itertuples() }

In [24]:
for CB in adata.obs['CB']:
    try: dict_CB_to_MetaData[CB]
    except KeyError: dict_CB_to_MetaData[CB] = [None, None, None]

In [25]:
adata.obs['CellLine'] = [ dict_CB_to_MetaData[CB][0] for CB in adata.obs['CB'] ]
adata.obs['PoolID'] = [ dict_CB_to_MetaData[CB][1] for CB in adata.obs['CB'] ]
adata.obs['CancerType'] = [ dict_CB_to_MetaData[CB][2] for CB in adata.obs['CB'] ]

In [27]:
adata.write('/node200data/18parkky/datasets/data/public/processed_data/Kinker_et_al/CPM_data.metalabeled.h5ad')

## Joanito et al (MSI)

In [2]:
joanito_metadata = pd.read_csv('/node200data/18parkky/datasets/data/public/processed_data/Joanito_et_al/CRC-SG1/synapse-metadata/patient_clinical_information.csv', encoding = "ISO-8859-1")
joanito_metadata = joanito_metadata[joanito_metadata['dataset'].isin(['CRC-SG1', 'KUL5', 'KUL3'])].copy()
joanito_metadata = joanito_metadata[joanito_metadata['MSS_MSI']=='MSI-H'].copy()

PATH_SG1_metadata = "/node200data/18parkky/datasets/data/public/processed_data/Joanito_et_al/CRC-SG1/EGAD00001008555-metadata/organized_metadata.tsv"
PATH_KUL_metadata = "/node200data/18parkky/datasets/data/public/processed_data/Joanito_et_al/KUL/organized_metadata.tsv"

SG1_metadata = pd.read_csv(PATH_SG1_metadata, sep='\t')
KUL_metadata = pd.read_csv(PATH_KUL_metadata, sep='\t')

SG1_metadata = SG1_metadata[SG1_metadata['patient.ID'].isin( joanito_metadata['patient.ID'] )].copy()
KUL_metadata = KUL_metadata[KUL_metadata['patient.ID'].isin( joanito_metadata['patient.ID'] )].copy()

print(f'{len(set(SG1_metadata['patient.ID']))} MSI-H patients from:\tCRC-SG1 cohort')
print(f'{len(set(KUL_metadata['patient.ID']))} MSI-H patients from:\tKUL cohort')

2 MSI-H patients from:	CRC-SG1 cohort
2 MSI-H patients from:	KUL cohort


In [3]:
DIR_CRC_SG1_cellranger_out = '/node200data/18parkky/datasets/data/public/processed_data/Joanito_et_al/CRC-SG1/cellranger'       #
DIR_KUL_MSI_cellranger_out = '/node200data/18parkky/datasets/data/public/processed_data/Joanito_et_al/KUL/MSI_H/cellranger'     # iCMS3 MSI

#### CRC-SG1 cohort

In [4]:
adata = list()

for patientID, edf in SG1_metadata.groupby("patient.ID"):
    print(patientID, flush=True)
    
    for tup in edf.itertuples():
        PATH_cellranger_h5ad = f'{DIR_CRC_SG1_cellranger_out}/{tup.file_ID}/outs/filtered_feature_bc_matrix.h5'
        PATH_cellranger_scrublet_out = f'{DIR_CRC_SG1_cellranger_out}/{tup.file_ID}/outs/filtered_feature_bc_matrix.scrublet.h5'
        
        if os.path.exists( PATH_cellranger_h5ad ) == True:
            if os.path.exists( PATH_cellranger_scrublet_out ) == True:  # Add to list
                adata_temp = sc.read_h5ad( PATH_cellranger_scrublet_out )
                
                adata_temp.obs['SampleID']  = tup.file_ID
                adata_temp.obs['PatientID'] = list(tup)[2]
                adata_temp.obs['BiopsySite'] = list(tup)[1].split("_")[1]
    
                adata.append( adata_temp ) 
                
            else:   # Run scrublet, save to disk then add to list
                adata_temp = sc.read_10x_h5( PATH_cellranger_h5ad )
                adata_temp.obs_names_make_unique()
                adata_temp.var_names_make_unique()
                sc.pp.scrublet(adata_temp)
                adata_temp.write(PATH_cellranger_scrublet_out)
                adata.append( adata_temp )
        else:   # Report
            print(f'{patientID}-{tup.file_ID} has not been processed by CellRanger!')

CRC2783
CRC2817


In [5]:
adata = sc.concat( adata  )
adata.obs_names_make_unique()

# mitochondrial genes, "MT-" for human, "Mt-" for mouse
adata.var["mt"] = adata.var_names.str.startswith("MT-")
# # ribosomal genes
# adata.var["ribo"] = adata.var_names.str.startswith(("RPS", "RPL"))
# # hemoglobin genes
# adata.var["hb"] = adata.var_names.str.contains("^HB[^(P)]")

sc.pp.calculate_qc_metrics(
    adata, qc_vars=["mt",], inplace=True, log1p=True
)

/home/18parkky/anaconda3-2021.11/envs/sc/lib/python3.12/site-packages/anndata/_core/anndata.py:1818: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")


In [8]:
PATH_adata_save = '/node200data/18parkky/datasets/data/public/processed_data/Joanito_et_al/1_raw_h5ad/Joanito.CRC-SG1.MSI.raw.h5ad'
adata.write(PATH_adata_save)

In [40]:
# adata = sc.read_h5ad(PATH_adata_save)

#### KUL cohort

In [55]:
KUL_metadata

,sample_alias,patient.ID,file_ID,Gender,MSS_MSI,Site,Sidedness,Group_Stage,Stage_TNM,iCMS_transcriptomic,iCMS_inferCNV,KRAS,BRAF,TP53,APC,PIK3CA,LymphNode,Normal,Tumor,CMS
0,SC035_tumor1,SC035,EXT048,M,MSI-H,Ascending colon,R,II,T3N0M0,iCMS3,iCMS3,wt,wt,wt,mut,NaN,1.0,1.0,2.0,CMS1
1,SC035_tumor2,SC035,EXT049,M,MSI-H,Ascending colon,R,II,T3N0M0,iCMS3,iCMS3,wt,wt,wt,mut,NaN,1.0,1.0,2.0,CMS1
2,SC035_normal,SC035,EXT050,M,MSI-H,Ascending colon,R,II,T3N0M0,iCMS3,iCMS3,wt,wt,wt,mut,NaN,1.0,1.0,2.0,CMS1
3,SC035_lymphnode,SC035,EXT051,M,MSI-H,Ascending colon,R,II,T3N0M0,iCMS3,iCMS3,wt,wt,wt,mut,NaN,1.0,1.0,2.0,CMS1
20,SC044_tumor1,SC044,EXT124,F,MSI-H,Caecum,R,III,T3N1M0,iCMS3,iCMS3,wt,wt,wt,wt,NaN,0.0,2.0,3.0,CMS1
21,SC044_tumor2,SC044,EXT125,F,MSI-H,Caecum,R,III,T3N1M0,iCMS3,iCMS3,wt,wt,wt,wt,NaN,0.0,2.0,3.0,CMS1
22,SC044_normal1,SC044,EXT126,F,MSI-H,Caecum,R,III,T3N1M0,iCMS3,iCMS3,wt,wt,wt,wt,NaN,0.0,2.0,3.0,CMS1
23,SC044_tumor3,SC044,EXT127,F,MSI-H,Caecum,R,III,T3N1M0,iCMS3,iCMS3,wt,wt,wt,wt,NaN,0.0,2.0,3.0,CMS1
24,SC044_tumor4,SC044,EXT128,F,MSI-H,Caecum,R,III,T3N1M0,iCMS3,iCMS3,wt,wt,wt,wt,NaN,0.0,2.0,3.0,CMS1
25,SC044_normal2,SC044,EXT129,F,MSI-H,Caecum,R,III,T3N1M0,iCMS3,iCMS3,wt,wt,wt,wt,NaN,0.0,2.0,3.0,CMS1


In [9]:
adata = list()

for patientID, edf in KUL_metadata.groupby("patient.ID"):
    print(patientID, flush=True)
    
    for tup in edf.itertuples():
        PATH_cellranger_h5ad = f'{DIR_KUL_MSI_cellranger_out}/{tup.file_ID}/outs/filtered_feature_bc_matrix.h5'
        PATH_cellranger_scrublet_out = f'{DIR_KUL_MSI_cellranger_out}/{tup.file_ID}/outs/filtered_feature_bc_matrix.scrublet.h5'
        
        if os.path.exists( PATH_cellranger_h5ad ) == True:
            if os.path.exists( PATH_cellranger_scrublet_out ) == True:  # Add to list
                adata_temp = sc.read_h5ad( PATH_cellranger_scrublet_out )
                
                adata_temp.obs['SampleID']  = tup.file_ID
                adata_temp.obs['PatientID'] = list(tup)[2]
                adata_temp.obs['BiopsySite'] = list(tup)[1].split("_")[1]
    
                adata.append( adata_temp ) 
                
            else:   # Run scrublet, save to disk then add to list
                print(f'{patientID}-{tup.file_ID} has not been processed by Scrublet! Running Scrublet now..')
                adata_temp = sc.read_10x_h5( PATH_cellranger_h5ad )
                adata_temp.obs_names_make_unique()
                adata_temp.var_names_make_unique()
                sc.pp.scrublet(adata_temp)
                adata_temp.write(PATH_cellranger_scrublet_out)
                adata.append( adata_temp )
        else:   # Report
            print(f'{patientID}-{tup.file_ID} has not been processed by CellRanger!')

SC035
SC044


In [10]:
adata = sc.concat( adata  )
adata.obs_names_make_unique()

# mitochondrial genes, "MT-" for human, "Mt-" for mouse
adata.var["mt"] = adata.var_names.str.startswith("MT-")
# # ribosomal genes
# adata.var["ribo"] = adata.var_names.str.startswith(("RPS", "RPL"))
# # hemoglobin genes
# adata.var["hb"] = adata.var_names.str.contains("^HB[^(P)]")

sc.pp.calculate_qc_metrics(
    adata, qc_vars=["mt",], inplace=True, log1p=True
)

/home/18parkky/anaconda3-2021.11/envs/sc/lib/python3.12/site-packages/anndata/_core/anndata.py:1818: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")


In [11]:
adata.obs.rename({'Sample' : "Accession"}, axis=1, inplace=True)
adata.obs['BiopsySite'] = adata.obs['BiopsySite'].map( {
                                                        'lymphnode' : 'LymphNode', 
                                                        'normal' : 'Adjacent normal tissue', 
                                                        'normal1' : 'Adjacent normal tissue',
                                                        'normal2' : 'Adjacent normal tissue',
                                                        'tumor1' : 'Tumor',
                                                        'tumor2' : 'Tumor',
                                                        'tumor3' : 'Tumor',
                                                        'tumor4' : 'Tumor',
                                                        },)

In [12]:
PATH_adata_save = '/node200data/18parkky/datasets/data/public/processed_data/Joanito_et_al/1_raw_h5ad/Joanito.KUL.MSI.raw.h5ad'
adata.write(PATH_adata_save)

In [45]:
# adata = sc.read_h5ad(PATH_adata_save)

## Joanito et al (MSS; 6 patients)

### CRC-SG1 cohort

In [ ]:
# iCMS2: CRC2801
# iCMS3: CRC2786, CRC2787, CRC2821

In [13]:
joanito_metadata = pd.read_csv('/node200data/18parkky/datasets/data/public/processed_data/Joanito_et_al/CRC-SG1/synapse-metadata/patient_clinical_information.csv', encoding = "ISO-8859-1")
joanito_metadata = joanito_metadata[joanito_metadata['dataset'].isin(['CRC-SG1', 'KUL5', 'KUL3'])].copy()
joanito_metadata = joanito_metadata[joanito_metadata['MSS_MSI']!='MSI-H'].copy()

PATH_SG1_metadata = "/node200data/18parkky/datasets/data/public/processed_data/Joanito_et_al/CRC-SG1/EGAD00001008555-metadata/organized_metadata.tsv"
PATH_KUL_metadata = "/node200data/18parkky/datasets/data/public/processed_data/Joanito_et_al/KUL/organized_metadata.tsv"

SG1_metadata = pd.read_csv(PATH_SG1_metadata, sep='\t')
KUL_metadata = pd.read_csv(PATH_KUL_metadata, sep='\t')

SG1_metadata = SG1_metadata[SG1_metadata['patient.ID'].isin( joanito_metadata['patient.ID'] )].copy()
KUL_metadata = KUL_metadata[KUL_metadata['patient.ID'].isin( joanito_metadata['patient.ID'] )].copy()

print(f'{len(set(SG1_metadata['patient.ID']))} MSS patients from:\tCRC-SG1 cohort')
print(f'{len(set(KUL_metadata['patient.ID']))} MSS patients from:\tKUL cohort')

12 MSS patients from:	CRC-SG1 cohort
3 MSS patients from:	KUL cohort


In [14]:
DIR_CRC_SG1_cellranger_out = '/node200data/18parkky/datasets/data/public/processed_data/Joanito_et_al/CRC-SG1/cellranger'       #
DIR_KUL_MSS_cellranger_out = '/node200data/18parkky/datasets/data/public/processed_data/Joanito_et_al/KUL/iCMS3_MSS'            # iCMS3 MSS

In [39]:
MSS_patients = list()

for patientID, edf in SG1_metadata.groupby("patient.ID"):
    boolFullyProcessed = True
    
    for tup in edf.itertuples():
        PATH_cellranger_h5ad = f'{DIR_CRC_SG1_cellranger_out}/{tup.file_ID}/outs/filtered_feature_bc_matrix.h5'
        PATH_cellranger_scrublet_out = f'{DIR_CRC_SG1_cellranger_out}/{tup.file_ID}/outs/filtered_feature_bc_matrix.scrublet.h5'
        
        if os.path.exists( PATH_cellranger_h5ad ) == True:
            pass 

        else:   # Report
            boolFullyProcessed = False 
            break 
    
    if boolFullyProcessed==True:
        print(patientID, boolFullyProcessed, edf.iloc[0].iCMS_transcriptomic, flush=True)
        MSS_patients.append( patientID )


CRC2786 True iCMS3
CRC2787 True iCMS3
CRC2801 True iCMS2
CRC2821 True iCMS3


In [16]:
adata = list()

for patientID, edf in SG1_metadata[SG1_metadata['patient.ID'].isin(MSS_patients)].groupby("patient.ID"):
    print(patientID, edf.iloc[0].iCMS_transcriptomic, flush=True)
    
    for tup in edf.itertuples():
        PATH_cellranger_h5ad = f'{DIR_CRC_SG1_cellranger_out}/{tup.file_ID}/outs/filtered_feature_bc_matrix.h5'
        PATH_cellranger_scrublet_out = f'{DIR_CRC_SG1_cellranger_out}/{tup.file_ID}/outs/filtered_feature_bc_matrix.scrublet.h5'
        
        if os.path.exists( PATH_cellranger_h5ad ) == True:
            if os.path.exists( PATH_cellranger_scrublet_out ) == True:  # Add to list
                adata_temp = sc.read_h5ad( PATH_cellranger_scrublet_out )
                
                adata_temp.obs['SampleID']  = tup.file_ID
                adata_temp.obs['PatientID'] = list(tup)[2]
                adata_temp.obs['BiopsySite'] = list(tup)[1].split("_")[1]
                adata.append( adata_temp ) 
                
            else:   # Run scrublet, save to disk then add to list
                adata_temp = sc.read_10x_h5( PATH_cellranger_h5ad )
                adata_temp.obs_names_make_unique()
                adata_temp.var_names_make_unique()
                sc.pp.scrublet(adata_temp)
                adata_temp.write(PATH_cellranger_scrublet_out)
                adata.append( adata_temp )
        else:   # Report
            print(f'{patientID}-{tup.file_ID} has not been processed by CellRanger!')

CRC2786 iCMS3
CRC2787 iCMS3
CRC2801 iCMS2
CRC2821 iCMS3


In [17]:
adata = sc.concat( adata  )
adata.obs_names_make_unique()

# mitochondrial genes, "MT-" for human, "Mt-" for mouse
adata.var["mt"] = adata.var_names.str.startswith("MT-")
# # ribosomal genes
# adata.var["ribo"] = adata.var_names.str.startswith(("RPS", "RPL"))
# # hemoglobin genes
# adata.var["hb"] = adata.var_names.str.contains("^HB[^(P)]")

sc.pp.calculate_qc_metrics(
    adata, qc_vars=["mt", ], inplace=True, log1p=True
)

/home/18parkky/anaconda3-2021.11/envs/sc/lib/python3.12/site-packages/anndata/_core/anndata.py:1818: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")


In [18]:
PATH_adata_save = '/node200data/18parkky/datasets/data/public/processed_data/Joanito_et_al/1_raw_h5ad/Joanito.CRC-SG1.MSS.raw.h5ad'
adata.write(PATH_adata_save)

### KUL cohort

In [ ]:
# iCMS3: SC040, SC041

In [38]:
KUL_metadata

,sample_alias,patient.ID,file_ID,Gender,MSS_MSI,Site,Sidedness,Group_Stage,Stage_TNM,iCMS_transcriptomic,iCMS_inferCNV,KRAS,BRAF,TP53,APC,PIK3CA,LymphNode,Normal,Tumor,CMS
4,SC040_tumor1,SC040,EXT093,F,MSS,Ascending colon,R,II,T3N0M0,iCMS3,iCMS3,mut,wt,wt,mut,NaN,1.0,2.0,4.0,NaN
5,SC040_tumor2,SC040,EXT094,F,MSS,Ascending colon,R,II,T3N0M0,iCMS3,iCMS3,mut,wt,wt,mut,NaN,1.0,2.0,4.0,NaN
6,SC040_normal1,SC040,EXT095,F,MSS,Ascending colon,R,II,T3N0M0,iCMS3,iCMS3,mut,wt,wt,mut,NaN,1.0,2.0,4.0,NaN
7,SC040_lymphnode,SC040,EXT096,F,MSS,Ascending colon,R,II,T3N0M0,iCMS3,iCMS3,mut,wt,wt,mut,NaN,1.0,2.0,4.0,NaN
8,SC040_tumor3,SC040,EXT097,F,MSS,Ascending colon,R,II,T3N0M0,iCMS3,iCMS3,mut,wt,wt,mut,NaN,1.0,2.0,4.0,NaN
9,SC040_tumor4,SC040,EXT098,F,MSS,Ascending colon,R,II,T3N0M0,iCMS3,iCMS3,mut,wt,wt,mut,NaN,1.0,2.0,4.0,NaN
10,SC040_normal2,SC040,EXT099,F,MSS,Ascending colon,R,II,T3N0M0,iCMS3,iCMS3,mut,wt,wt,mut,NaN,1.0,2.0,4.0,NaN
11,SC041_tumor1,SC041,EXT113,M,MSS,Rectosigmoid,L,III,T4N1M0,iCMS3,iCMS3,mut,mut,wt,mut,NaN,0.0,1.0,2.0,CMS3
12,SC041_tumor2,SC041,EXT114,M,MSS,Rectosigmoid,L,III,T4N1M0,iCMS3,iCMS3,mut,mut,wt,mut,NaN,0.0,1.0,2.0,CMS3
13,SC041_normal,SC041,EXT115,M,MSS,Rectosigmoid,L,III,T4N1M0,iCMS3,iCMS3,mut,mut,wt,mut,NaN,0.0,1.0,2.0,CMS3


In [39]:
DIR_KUL_MSI_cellranger_out

'/node200data/18parkky/datasets/data/public/processed_data/Joanito_et_al/KUL/iCMS3_MSS'

In [19]:
MSS_patients = list()

for patientID, edf in KUL_metadata.groupby("patient.ID"):
    boolFullyProcessed = True
    
    for tup in edf.itertuples():
        PATH_cellranger_h5ad = f'{DIR_KUL_MSS_cellranger_out}/{tup.file_ID}/outs/filtered_feature_bc_matrix.h5'
        PATH_cellranger_scrublet_out = f'{DIR_KUL_MSS_cellranger_out}/{tup.file_ID}/outs/filtered_feature_bc_matrix.scrublet.h5'
        
        if os.path.exists( PATH_cellranger_h5ad ) == True:
            pass 

        else:   # Report
            boolFullyProcessed = False 
            break 
    
    if boolFullyProcessed==True:
        print(patientID, boolFullyProcessed, edf.iloc[0].iCMS_transcriptomic, flush=True)
        MSS_patients.append( patientID )

SC040 True iCMS3
SC041 True iCMS3


In [20]:
adata = list()

for patientID, edf in KUL_metadata[KUL_metadata['patient.ID'].isin(MSS_patients)].groupby("patient.ID"):
    print(patientID, flush=True)
    
    for tup in edf.itertuples():
        PATH_cellranger_h5ad = f'{DIR_KUL_MSS_cellranger_out}/{tup.file_ID}/outs/filtered_feature_bc_matrix.h5'
        PATH_cellranger_scrublet_out = f'{DIR_KUL_MSS_cellranger_out}/{tup.file_ID}/outs/filtered_feature_bc_matrix.scrublet.h5'
        
        if os.path.exists( PATH_cellranger_h5ad ) == True:
            if os.path.exists( PATH_cellranger_scrublet_out ) == True:  # Add to list
                adata_temp = sc.read_h5ad( PATH_cellranger_scrublet_out )
                
                adata_temp.obs['SampleID']  = tup.file_ID
                adata_temp.obs['PatientID'] = list(tup)[2]
                adata_temp.obs['BiopsySite'] = list(tup)[1].split("_")[1]
    
                adata.append( adata_temp ) 
                
            else:   # Run scrublet, save to disk then add to list
                print(f'{patientID}-{tup.file_ID} has not been processed by Scrublet! Running Scrublet now..')
                adata_temp = sc.read_10x_h5( PATH_cellranger_h5ad )
                adata_temp.obs_names_make_unique()
                adata_temp.var_names_make_unique()
                sc.pp.scrublet(adata_temp)
                adata_temp.write(PATH_cellranger_scrublet_out)
                adata.append( adata_temp )
        else:   # Report
            print(f'{patientID}-{tup.file_ID} has not been processed by CellRanger!')

SC040
SC041


In [21]:
adata = sc.concat( adata  )
adata.obs_names_make_unique()

# mitochondrial genes, "MT-" for human, "Mt-" for mouse
adata.var["mt"] = adata.var_names.str.startswith("MT-")
# # ribosomal genes
# adata.var["ribo"] = adata.var_names.str.startswith(("RPS", "RPL"))
# # hemoglobin genes
# adata.var["hb"] = adata.var_names.str.contains("^HB[^(P)]")

sc.pp.calculate_qc_metrics(
    adata, qc_vars=["mt", ], inplace=True, log1p=True
)

/home/18parkky/anaconda3-2021.11/envs/sc/lib/python3.12/site-packages/anndata/_core/anndata.py:1818: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")


In [22]:
adata.obs.rename({'Sample' : "Accession"}, axis=1, inplace=True)
adata.obs['BiopsySite'] = adata.obs['BiopsySite'].map( {
                                                        'lymphnode' : 'LymphNode', 
                                                        'normal' : 'Adjacent normal tissue', 
                                                        'normal1' : 'Adjacent normal tissue',
                                                        'normal2' : 'Adjacent normal tissue',
                                                        'tumor1' : 'Tumor',
                                                        'tumor2' : 'Tumor',
                                                        'tumor3' : 'Tumor',
                                                        'tumor4' : 'Tumor',
                                                        },)

In [23]:
PATH_adata_save = '/node200data/18parkky/datasets/data/public/processed_data/Joanito_et_al/1_raw_h5ad/Joanito.KUL.MSS.raw.h5ad'
adata.write(PATH_adata_save)

## Chen et al

In [7]:
chen_metadata = pd.read_csv('/node200data/18parkky/datasets/data/public/FASTQ/Immunotherapy_CRC_Chen_et_al/metadata/Chen_et_al.MSI.run_meta.organized.tsv', sep='\t')
chen_metadata = chen_metadata[['Accession', 'SampleID', 'PatientID', 'TreatmentStage', 'TreatmentPoint', 'BiopsySite', 'MMR', 'MSI', 'TMB', 'Response', 'LibraryType']].copy()
chen_metadata = chen_metadata[(chen_metadata['LibraryType']=='scRNA-seq')].copy()
chen_metadata = chen_metadata[(chen_metadata['BiopsySite'].isin( ['Adjacent normal tissue', 'Tumor'] ))].copy()
chen_metadata['BiopsySite'] = chen_metadata['BiopsySite'].map( { 
                                                                'Adjacent normal tissue' : 'Normal', 
                                                                'Tumor' : 'Tumor', 
                                                                })

chen_patient_metadata = pd.read_csv('/node200data/18parkky/datasets/data/public/FASTQ/Immunotherapy_CRC_Chen_et_al/metadata/Chen_et_al.patient_meta.csv',)
dict_PatientID_to_TumorLoc   = { list(tup)[1] : list(tup)[5] for tup in chen_patient_metadata.itertuples() }
dict_PatientID_to_TumorLoc['P15'] = 'Sigmoid colon'
dict_PatientID_to_CancerType = { list(tup)[1] : list(tup)[4] for tup in chen_patient_metadata.itertuples() }

chen_metadata['TumorLocation'] = [ dict_PatientID_to_TumorLoc[PID] for PID in chen_metadata['PatientID'] ]
chen_metadata['CancerType'] = [ dict_PatientID_to_CancerType[PID] for PID in chen_metadata['PatientID'] ]

chen_metadata = chen_metadata[(chen_metadata['CancerType']=='CRC') & (chen_metadata['Response']!='SD')]

print(f'{len(set(chen_metadata['PatientID']))} MSI-H patients from:\tChen et al. cohort')
chen_metadata.to_csv('/node200data/18parkky/datasets/data/public/FASTQ/Immunotherapy_CRC_Chen_et_al/metadata/Chen_et_al.MSI.CRC.woSD.run_meta.organized.tsv', sep='\t', index=False)

12 MSI-H patients from:	Chen et al. cohort


### 1. Run Cell ranger of each sample (input mutliple FASTQ of same sample)

In [9]:
len(set(chen_metadata['SampleID'])), len(set(chen_metadata[chen_metadata['TreatmentStage']=='Pre']['SampleID']))

(64, 24)

In [10]:
PATH_cellranger_count_bin       = '/home/shared/program/cellranger-8.0.1/bin/cellranger'
PATH_cellranger_transcriptome   = '/home/shared/cellranger_reference/refdata-gex-GRCh38-2020-A'

In [8]:
# Total number of samples we need to process
len(set(chen_metadata[(chen_metadata['TreatmentStage']=='Pre') & (chen_metadata['BiopsySite']=='Tumor')]['SampleID']))

12

In [28]:
# Tumor
DIR_merged_cellranger_out = '/node200data/18parkky/datasets/data/public/BAM/Immunotherapy_CRC_Chen_et_al/MSI/Tumor_merged'
DIR_FASTQ_out = '/node200data/18parkky/datasets/data/public/FASTQ/Immunotherapy_CRC_Chen_et_al/MSI/Tumor'

run_cellranger_sh   = open(f'{DIR_merged_cellranger_out}/run_celleranger.sh', 'w')
run_download_fq_sh  = open(f'{DIR_FASTQ_out}/download_fastq.sh', 'w')

for SampleID, edf in chen_metadata[(chen_metadata['TreatmentStage']=='Pre') & (chen_metadata['BiopsySite']=='Tumor')].groupby("SampleID"):
    DIR_FASTQ = '/node200data/18parkky/datasets/data/public/FASTQ/Immunotherapy_CRC_Chen_et_al/MSI/Tumor'
    allFastqExists = True
    list_PATH_FASTQ = list()
    for tup in edf.itertuples():
        PATH_FASTQ = f'{DIR_FASTQ}/{tup.Accession}'
        if os.path.exists(PATH_FASTQ)==False:
            # print(f'{SampleID}\t{tup.Accession}')
            allFastqExists = False 
            break 
        else:
            list_PATH_FASTQ.append( PATH_FASTQ )

    if allFastqExists == True:
        PATH_merged_cellranger_out = f'{DIR_merged_cellranger_out}/{SampleID}'
        if os.path.exists(PATH_merged_cellranger_out)==True:
            continue 
        cmd = f'{PATH_cellranger_count_bin} count --id {SampleID} --transcriptome {PATH_cellranger_transcriptome} --fastqs {",".join(list_PATH_FASTQ)} --create-bam true --nosecondary --output-dir {DIR_merged_cellranger_out}/{SampleID}'
        print(cmd)
        run_cellranger_sh.write(f'{cmd}\n')
    else:
        print(f'FASTQ not intact: {SampleID} ({len(list_PATH_FASTQ)}/{len(edf)} found)')
        for tup in edf.itertuples():
            PATH_FASTQ = f'{DIR_FASTQ}/{tup.Accession}'
            if os.path.exists(PATH_FASTQ)==False:
                print(f'\t{SampleID}\t{tup.Accession}')
                run_download_fq_sh.write( f'wget ftp://human.big.ac.cn:/HRA005546/{tup.Accession}/{tup.Accession}_f1.fastq.gz -O {DIR_FASTQ_out}/{tup.Accession}/{tup.Accession}_S1_L001_R1_001.fastq.gz --ftp-user=\'jihwan.park@gist.ac.kr\' --ftp-password=\'Wlshalrtm_204!\'\n' )
                run_download_fq_sh.write( f'wget ftp://human.big.ac.cn:/HRA005546/{tup.Accession}/{tup.Accession}_r2.fastq.gz -O {DIR_FASTQ_out}/{tup.Accession}/{tup.Accession}_S1_L001_R2_001.fastq.gz --ftp-user=\'jihwan.park@gist.ac.kr\' --ftp-password=\'Wlshalrtm_204!\'\n' )

run_cellranger_sh.close()
run_download_fq_sh.close()

In [6]:
# NAT
DIR_merged_cellranger_out = '/node200data/18parkky/datasets/data/public/BAM/Immunotherapy_CRC_Chen_et_al/MSI/Adjacent_normal_tissue_merged'
DIR_FASTQ_out = '/node200data/18parkky/datasets/data/public/FASTQ/Immunotherapy_CRC_Chen_et_al/MSI/Adjacent_normal_tissue'

run_cellranger_sh   = open(f'{DIR_merged_cellranger_out}/run_celleranger.sh', 'w')
run_download_fq_sh  = open(f'{DIR_FASTQ_out}/download_fastq.sh', 'w')

# _S1_L001_R1_001
for SampleID, edf in chen_metadata[(chen_metadata['TreatmentStage']=='Pre') & (chen_metadata['BiopsySite']!='Tumor')].groupby("SampleID"):
    DIR_FASTQ = '/node200data/18parkky/datasets/data/public/FASTQ/Immunotherapy_CRC_Chen_et_al/MSI/Adjacent_normal_tissue'
    allFastqExists = True
    list_PATH_FASTQ = list()
    for tup in edf.itertuples():
        PATH_FASTQ = f'{DIR_FASTQ}/{tup.Accession}'
        if os.path.exists(PATH_FASTQ)==False:
            # print(f'{SampleID}\t{tup.Accession}')
            allFastqExists = False 
            break 
        else:
            list_PATH_FASTQ.append( PATH_FASTQ )

    if allFastqExists == True:
        PATH_merged_cellranger_out = f'{DIR_merged_cellranger_out}/{SampleID}'
        if os.path.exists(PATH_merged_cellranger_out)==True:
            continue 
        cmd = f'{PATH_cellranger_count_bin} count --id {SampleID} --transcriptome {PATH_cellranger_transcriptome} --fastqs {",".join(list_PATH_FASTQ)} --create-bam true --nosecondary --output-dir {DIR_merged_cellranger_out}/{SampleID}'
        run_cellranger_sh.write(f'{cmd}\n')

        print(cmd)
    else:
        print(f'FASTQ not intact: {SampleID} ({len(list_PATH_FASTQ)}/{len(edf)} found)')
        for tup in edf.itertuples():
            PATH_FASTQ = f'{DIR_FASTQ}/{tup.Accession}'
            if os.path.exists(PATH_FASTQ)==False:
                print(f'\t{SampleID}\t{tup.Accession}')
                run_download_fq_sh.write(f'mkdir {PATH_FASTQ}\n')
                run_download_fq_sh.write( f'wget ftp://human.big.ac.cn:/HRA005546/{tup.Accession}/{tup.Accession}_f1.fastq.gz -O {PATH_FASTQ}/{tup.Accession}_S1_L001_R1_001.fastq.gz --ftp-user=\'jihwan.park@gist.ac.kr\' --ftp-password=\'Wlshalrtm_204!\'\n' )
                run_download_fq_sh.write( f'wget ftp://human.big.ac.cn:/HRA005546/{tup.Accession}/{tup.Accession}_r2.fastq.gz -O {PATH_FASTQ}/{tup.Accession}_S1_L001_R2_001.fastq.gz --ftp-user=\'jihwan.park@gist.ac.kr\' --ftp-password=\'Wlshalrtm_204!\'\n' )

run_cellranger_sh.close()
run_download_fq_sh.close()

/home/shared/program/cellranger-8.0.1/bin/cellranger count --id P11-N-I --transcriptome /home/shared/cellranger_reference/refdata-gex-GRCh38-2020-A --fastqs /node200data/18parkky/datasets/data/public/FASTQ/Immunotherapy_CRC_Chen_et_al/MSI/Adjacent_normal_tissue/HRR1373268,/node200data/18parkky/datasets/data/public/FASTQ/Immunotherapy_CRC_Chen_et_al/MSI/Adjacent_normal_tissue/HRR1373267,/node200data/18parkky/datasets/data/public/FASTQ/Immunotherapy_CRC_Chen_et_al/MSI/Adjacent_normal_tissue/HRR1373266,/node200data/18parkky/datasets/data/public/FASTQ/Immunotherapy_CRC_Chen_et_al/MSI/Adjacent_normal_tissue/HRR1373265 --create-bam true --nosecondary --output-dir /node200data/18parkky/datasets/data/public/BAM/Immunotherapy_CRC_Chen_et_al/MSI/Adjacent_normal_tissue_merged/P11-N-I
/home/shared/program/cellranger-8.0.1/bin/cellranger count --id P17-N-I --transcriptome /home/shared/cellranger_reference/refdata-gex-GRCh38-2020-A --fastqs /node200data/18parkky/datasets/data/public/FASTQ/Immunother

### 2. Run NanoMnT

In [11]:
# Tumor
DIR_merged_cellranger_out = '/node200data/18parkky/datasets/data/public/BAM/Immunotherapy_CRC_Chen_et_al/MSI/Tumor_merged'
list_PATH_possorted_BAM = [ f'{DIR}/outs/possorted_genome_bam.bam' for DIR in glob.glob(f'{DIR_merged_cellranger_out}/*') if (os.path.exists(f'{DIR}/outs/possorted_genome_bam.bam')==True) and (os.path.exists(f'{DIR}/outs/possorted_genome_bam.STR_allele_table.tsv')==False) ]

PATH_krait = '/home/18parkky/git/NanoMnT/ref/krait/raw/GRCh38/GRCh38-1bp_STR.gene.tsv.gz'
PATH_ref_genome = '/node200data/18parkky/datasets/reference/genome/hg38.fa'
num_threads = 36
realignment_flanking = 1000 # no point in using 20000 bp flanking for short-read data 

# run_nanomnt_sh = open(f'{DIR_merged_cellranger_out}/run_nanomnt.sh', 'w')

for PATH_possorted_BAM in list_PATH_possorted_BAM:
    PATH_nanomnt_out = "/".join(PATH_possorted_BAM.split("/")[:-1])
    PATH_getAlleleTable_out     = f'{PATH_nanomnt_out}/possorted_genome_bam.STR_allele_table.tsv'
    DIR_getAlleleTable_tmpdir   = f'{PATH_nanomnt_out}/possorted_genome_bam.multiprocessing_temp'
    if os.path.exists(PATH_getAlleleTable_out)==False and os.path.exists(DIR_getAlleleTable_tmpdir)==False:
        cmd = f'getAlleleTable -b {PATH_possorted_BAM} -s {PATH_krait} -r {PATH_ref_genome} -t {num_threads} -f 12 -m 1 --sc -rf {realignment_flanking} -out {PATH_nanomnt_out}'
        # run_nanomnt_sh.write(f'{cmd}\n')
        print(cmd)

getAlleleTable -b /node200data/18parkky/datasets/data/public/BAM/Immunotherapy_CRC_Chen_et_al/MSI/Tumor_merged/P15-T-I/outs/possorted_genome_bam.bam -s /home/18parkky/git/NanoMnT/ref/krait/raw/GRCh38/GRCh38-1bp_STR.gene.tsv.gz -r /node200data/18parkky/datasets/reference/genome/hg38.fa -t 36 -f 12 -m 1 --sc -rf 1000 -out /node200data/18parkky/datasets/data/public/BAM/Immunotherapy_CRC_Chen_et_al/MSI/Tumor_merged/P15-T-I/outs
getAlleleTable -b /node200data/18parkky/datasets/data/public/BAM/Immunotherapy_CRC_Chen_et_al/MSI/Tumor_merged/P08-T-I/outs/possorted_genome_bam.bam -s /home/18parkky/git/NanoMnT/ref/krait/raw/GRCh38/GRCh38-1bp_STR.gene.tsv.gz -r /node200data/18parkky/datasets/reference/genome/hg38.fa -t 36 -f 12 -m 1 --sc -rf 1000 -out /node200data/18parkky/datasets/data/public/BAM/Immunotherapy_CRC_Chen_et_al/MSI/Tumor_merged/P08-T-I/outs
getAlleleTable -b /node200data/18parkky/datasets/data/public/BAM/Immunotherapy_CRC_Chen_et_al/MSI/Tumor_merged/P16-T-I/outs/possorted_genome_bam

### 2. Merge data 

In [11]:
adata = list()

In [12]:
chen_metadata

,Accession,SampleID,PatientID,TreatmentStage,TreatmentPoint,BiopsySite,MMR,MSI,TMB,Response,LibraryType,TumorLocation,CancerType
24,HRR1372976,P04-N-I,P04,Pre,Baseline,Normal,dMMR,MSI,487,CR,scRNA-seq,Ascending colon,CRC
25,HRR1372975,P04-N-I,P04,Pre,Baseline,Normal,dMMR,MSI,487,CR,scRNA-seq,Ascending colon,CRC
26,HRR1372974,P04-N-I,P04,Pre,Baseline,Normal,dMMR,MSI,487,CR,scRNA-seq,Ascending colon,CRC
27,HRR1372973,P04-N-I,P04,Pre,Baseline,Normal,dMMR,MSI,487,CR,scRNA-seq,Ascending colon,CRC
32,HRR1373024,P04-N-II,P04,On,Post 2 cycle,Normal,dMMR,MSI,487,CR,scRNA-seq,Ascending colon,CRC
...,...,...,...,...,...,...,...,...,...,...,...,...,...
979,HRR1373833,P26-T-I,P26,Pre,Baseline,Tumor,dMMR,MSI,75.85,CR,scRNA-seq,Ascending colon,CRC
984,HRR1374144,P26-T-II,P26,Post,Post 3 cycle,Tumor,dMMR,MSI,75.85,CR,scRNA-seq,Ascending colon,CRC
985,HRR1374143,P26-T-II,P26,Post,Post 3 cycle,Tumor,dMMR,MSI,75.85,CR,scRNA-seq,Ascending colon,CRC
986,HRR1374142,P26-T-II,P26,Post,Post 3 cycle,Tumor,dMMR,MSI,75.85,CR,scRNA-seq,Ascending colon,CRC


In [13]:
# NAT
DIR_merged_NAT_cellranger_out = '/node200data/18parkky/datasets/data/public/BAM/Immunotherapy_CRC_Chen_et_al/MSI/Adjacent_normal_tissue_merged'

for SampleID, edf in chen_metadata[(chen_metadata['TreatmentStage']=='Pre') & (chen_metadata['BiopsySite']=='Normal')].groupby('SampleID'):
    PATH_cellranger_h5ad = f'{DIR_merged_NAT_cellranger_out}/{SampleID}/outs/filtered_feature_bc_matrix.h5'
    PATH_cellranger_scrublet_out = f'{DIR_merged_NAT_cellranger_out}/{SampleID}/outs/filtered_feature_bc_matrix.scrublet.h5'

    if os.path.exists( PATH_cellranger_h5ad ) == True:
        if os.path.exists( PATH_cellranger_scrublet_out ) == True:  # Add to list
            adata_temp = sc.read_h5ad( PATH_cellranger_scrublet_out )
            
            adata_temp.obs['Accession'] = edf.iloc[0].Accession
            adata_temp.obs['SampleID']  = SampleID
            adata_temp.obs['PatientID'] = edf.iloc[0].PatientID
            adata_temp.obs['BiopsySite'] = edf.iloc[0].BiopsySite

            adata.append( adata_temp ) 
            pass
            
        else:   # Run scrublet, save to disk then add to list
            adata_temp = sc.read_10x_h5( PATH_cellranger_h5ad )
            adata_temp.obs_names_make_unique()
            adata_temp.var_names_make_unique()
            
            sc.pp.scrublet(adata_temp)
            
            adata_temp.obs['Accession'] = edf.iloc[0].Accession
            adata_temp.obs['SampleID']  = SampleID
            adata_temp.obs['PatientID'] = edf.iloc[0].PatientID
            adata_temp.obs['BiopsySite'] = edf.iloc[0].BiopsySite
            
            adata_temp.write(PATH_cellranger_scrublet_out)
            adata.append( adata_temp )
            pass
    else:   # Report
        print(f'{patientID}-{tup.Accession} has not been processed by CellRanger!')

In [14]:
# Tumor
DIR_merged_Tumor_cellranger_out = '/node200data/18parkky/datasets/data/public/BAM/Immunotherapy_CRC_Chen_et_al/MSI/Tumor_merged'

for SampleID, edf in chen_metadata[(chen_metadata['TreatmentStage']=='Pre') & (chen_metadata['BiopsySite']=='Tumor')].groupby('SampleID'):
    PATH_cellranger_h5ad = f'{DIR_merged_Tumor_cellranger_out}/{SampleID}/outs/filtered_feature_bc_matrix.h5'
    PATH_cellranger_scrublet_out = f'{DIR_merged_Tumor_cellranger_out}/{SampleID}/outs/filtered_feature_bc_matrix.scrublet.h5'

    if os.path.exists( PATH_cellranger_h5ad ) == True:
        if os.path.exists( PATH_cellranger_scrublet_out ) == True:  # Add to list
            adata_temp = sc.read_h5ad( PATH_cellranger_scrublet_out )
            
            adata_temp.obs['Accession'] = edf.iloc[0].Accession
            adata_temp.obs['SampleID']  = SampleID
            adata_temp.obs['PatientID'] = edf.iloc[0].PatientID
            adata_temp.obs['BiopsySite'] = edf.iloc[0].BiopsySite

            adata.append( adata_temp ) 
            pass
            
        else:   # Run scrublet, save to disk then add to list
            adata_temp = sc.read_10x_h5( PATH_cellranger_h5ad )
            adata_temp.obs_names_make_unique()
            adata_temp.var_names_make_unique()
            
            sc.pp.scrublet(adata_temp)
            
            adata_temp.obs['Accession'] = edf.iloc[0].Accession
            adata_temp.obs['SampleID']  = SampleID
            adata_temp.obs['PatientID'] = edf.iloc[0].PatientID
            adata_temp.obs['BiopsySite'] = edf.iloc[0].BiopsySite
            
            adata_temp.write(PATH_cellranger_scrublet_out)
            adata.append( adata_temp )
            pass
    else:   # Report
        print(f'{patientID}-{tup.Accession} has not been processed by CellRanger!')

In [15]:
adata = sc.concat( adata  )
adata.obs_names_make_unique()

# mitochondrial genes, "MT-" for human, "Mt-" for mouse
adata.var["mt"] = adata.var_names.str.startswith("MT-")
# # ribosomal genes
# adata.var["ribo"] = adata.var_names.str.startswith(("RPS", "RPL"))
# # hemoglobin genes
# adata.var["hb"] = adata.var_names.str.contains("^HB[^(P)]")

sc.pp.calculate_qc_metrics(
    adata, qc_vars=["mt",], inplace=True, log1p=True
)

/home/18parkky/anaconda3-2021.11/envs/sc/lib/python3.12/site-packages/anndata/_core/anndata.py:1818: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")


In [16]:
PATH_adata_save = '/node200data/18parkky/datasets/data/public/processed_data/Immunotherapy_CRC_Chen_et_al/1_raw_h5ad/Chen.MSI.raw.h5ad'
adata.write(PATH_adata_save)

In [ ]:
raise ValueError

In [ ]:
adata = list()
processed_sample_count = 0
unprocessed_sample_count = 0

for patientID, edf in chen_metadata.groupby('PatientID'):
    print(patientID, flush=True)
    for tup in edf.itertuples():
    
        if tup.BiopsySite == 'Normal':
            DIR_Chen_cellranger_out_temp = f'{DIR_Chen_cellranger_out}/Adjacent_normal_tissue'
        elif tup.BiopsySite == 'Tumor':
            DIR_Chen_cellranger_out_temp = f'{DIR_Chen_cellranger_out}/Tumor'
            
        PATH_cellranger_h5ad = f'{DIR_Chen_cellranger_out_temp}/{tup.Accession}/outs/filtered_feature_bc_matrix.h5'
        PATH_cellranger_scrublet_out = f'{DIR_Chen_cellranger_out_temp}/{tup.Accession}/outs/filtered_feature_bc_matrix.scrublet.h5'
        
        if os.path.exists( PATH_cellranger_h5ad ) == True:
            if os.path.exists( PATH_cellranger_scrublet_out ) == True:  # Add to list
                adata_temp = sc.read_h5ad( PATH_cellranger_scrublet_out )
                
                adata_temp.obs['SampleID']  = tup.Accession
                adata_temp.obs['PatientID'] = tup.PatientID
                adata_temp.obs['BiopsySite'] = tup.BiopsySite

                adata.append( adata_temp ) 
                pass
                
            else:   # Run scrublet, save to disk then add to list
                adata_temp = sc.read_10x_h5( PATH_cellranger_h5ad )
                adata_temp.obs_names_make_unique()
                adata_temp.var_names_make_unique()
                sc.pp.scrublet(adata_temp)
                adata_temp.write(PATH_cellranger_scrublet_out)
                adata.append( adata_temp )
                pass
        else:   # Report
            unprocessed_sample_count += 1
            print(f'{patientID}-{tup.Accession} has not been processed by CellRanger!')

In [ ]:
# adata = sc.concat( adata  )
# adata.obs_names_make_unique()

# # mitochondrial genes, "MT-" for human, "Mt-" for mouse
# adata.var["mt"] = adata.var_names.str.startswith("MT-")
# # ribosomal genes
# adata.var["ribo"] = adata.var_names.str.startswith(("RPS", "RPL"))
# # hemoglobin genes
# adata.var["hb"] = adata.var_names.str.contains("^HB[^(P)]")

# sc.pp.calculate_qc_metrics(
#     adata, qc_vars=["mt", "ribo", "hb"], inplace=True, log1p=True
# )

In [ ]:
# adata.obs['Accession'] = adata.obs['SampleID']

In [ ]:
# adata.write('/node200data/18parkky/datasets/data/public/processed_data/Immunotherapy_CRC_Chen_et_al/1_raw_h5ad/Chen.MSI.raw.h5ad')

In [ ]:
raise ValueError

## Depr (Chen)

#### Optional) Summarize sample preprocessing status

In [49]:
DIR_Chen_cellranger_out = '/node200data/18parkky/datasets/data/public/BAM/Immunotherapy_CRC_Chen_et_al/MSI'

In [116]:
sample_status_summary = list()

for patientID, edf in chen_metadata.groupby('PatientID'):
    for tup in edf.itertuples():
    
        if tup.BiopsySite == 'Normal':
            DIR_Chen_cellranger_out_temp = f'{DIR_Chen_cellranger_out}/Adjacent_normal_tissue'
        elif tup.BiopsySite == 'Tumor':
            DIR_Chen_cellranger_out_temp = f'{DIR_Chen_cellranger_out}/Tumor'
        else: raise ValueError    
        
        PATH_cellranger_h5ad = f'{DIR_Chen_cellranger_out_temp}/{tup.Accession}/outs/filtered_feature_bc_matrix.h5'
        PATH_cellranger_scrublet_out = f'{DIR_Chen_cellranger_out_temp}/{tup.Accession}/outs/filtered_feature_bc_matrix.scrublet.h5'
        
        if os.path.exists( PATH_cellranger_h5ad ) == True:
            bool_cellranger = True
            if os.path.exists( PATH_cellranger_scrublet_out ) == True:
                bool_scrublet = True
            else:
                bool_scrublet = False
            
            if os.path.exists( f'{DIR_Chen_cellranger_out_temp}/{tup.Accession}/outs/possorted_genome_bam.STR_allele_table.tsv' ):
                bool_nanomnt = True 
            else:
                bool_nanomnt = False 
                
        else:
            bool_cellranger = False
            bool_scrublet = False 
            bool_nanomnt = False
        
        sample_status_summary.append( [tup.Accession, tup.PatientID, tup.BiopsySite, bool_cellranger, bool_scrublet, bool_nanomnt] )

sample_status_summary = pd.DataFrame(sample_status_summary, columns=['Accession', 'PatientID', 'BiopsySite', 'cellranger', 'scrublet', 'nanomnt'])

#### Optional) Run cellranger for samples that have not been processed (create bash scripts)

In [10]:
PATH_cellranger_bin = '/home/shared/program/cellranger-8.0.1/bin/cellranger'
PATH_cellranger_ref = '/home/shared/cellranger_reference/refdata-gex-GRCh38-2020-A'

for BiopsySite, edf in sample_status_summary[sample_status_summary['cellranger']==False].groupby("BiopsySite"):
    with open(f'/node200data/18parkky/datasets/data/public/BAM/Immunotherapy_CRC_Chen_et_al/MSI/{BiopsySite}.run_cellranger.sh', 'w') as sh:
        for tup in edf.itertuples():

            if BiopsySite == 'Normal':
                DIR_Chen_cellranger_out_temp = f'/node200data/18parkky/datasets/data/public/BAM/Immunotherapy_CRC_Chen_et_al/MSI/Adjacent_normal_tissue/{tup.Accession}'
                DIR_Chen_FASTQ_temp = f'/node200data/18parkky/datasets/data/public/FASTQ/Immunotherapy_CRC_Chen_et_al/MSI/Adjacent_normal_tissue/{tup.Accession}'
            elif BiopsySite == 'Tumor':
                DIR_Chen_cellranger_out_temp = f'/node200data/18parkky/datasets/data/public/BAM/Immunotherapy_CRC_Chen_et_al/MSI/Tumor/{tup.Accession}'
                DIR_Chen_FASTQ_temp = f'/node200data/18parkky/datasets/data/public/FASTQ/Immunotherapy_CRC_Chen_et_al/MSI/Tumor/{tup.Accession}'
            else: raise ValueError
            
            if os.path.exists(DIR_Chen_cellranger_out_temp):
                print(f'Cell Ranger output directory already exists for: {BiopsySite}-{tup.Accession}')
                continue
            if os.path.exists(DIR_Chen_FASTQ_temp) == False:
                print(f'Cannot find FASTQ directory for: {BiopsySite}-{tup.Accession}')
                continue
            elif len( glob.glob(f'{DIR_Chen_FASTQ_temp}/*.fastq.gz') ) != 2:
                print(f'Cannot locate 2 FASTQ files for: {BiopsySite}-{tup.Accession}')
                continue

            cmd = f'echo {tup.Accession}'
            sh.write(f'{cmd}\n')
            cmd = f'{PATH_cellranger_bin} count --id {tup.Accession} --transcriptome {PATH_cellranger_ref} --fastqs {DIR_Chen_FASTQ_temp} --create-bam true --output-dir {DIR_Chen_cellranger_out_temp}'
            sh.write(f'{cmd}\n')

Cell Ranger output directory already exists for: Normal-HRR1373904


#### Optional) Run Scrublet to samples that have not been processed

In [92]:
for tup in sample_status_summary[(sample_status_summary['cellranger']==True) & (sample_status_summary['scrublet']==False)].itertuples():
    if tup.BiopsySite == 'Normal':
        PATH_adata_e = f'{DIR_Chen_cellranger_out}/Adjacent_normal_tissue/{tup.Accession}/outs/filtered_feature_bc_matrix.h5'
        PATH_scrublet_adata_e = f'{DIR_Chen_cellranger_out}/Adjacent_normal_tissue/{tup.Accession}/outs/filtered_feature_bc_matrix.scrublet.h5'
    elif tup.BiopsySite == 'Tumor':
        PATH_adata_e = f'{DIR_Chen_cellranger_out}/Tumor/{tup.Accession}/outs/filtered_feature_bc_matrix.h5'
        PATH_scrublet_adata_e = f'{DIR_Chen_cellranger_out}/Tumor/{tup.Accession}/outs/filtered_feature_bc_matrix.scrublet.h5'
    else:
        print(f'Unidentified Biopsy site for {tup.Accession}: {tup.BiopsySite}')
        raise ValueError
    
    if os.path.exists(PATH_adata_e)==False:
        print(f'Cell Ranger has been processed for: {tup.Accession}')
        raise ValueError
    if os.path.exists(PATH_scrublet_adata_e)==True:
        print(f'WARNING: Scrublet has been processed for: {tup.Accession}')
        print(f'WARNING: Skipping\n')
        continue 
    else:
        adata_e = sc.read_10x_h5( PATH_adata_e )
        adata_e.obs_names_make_unique()
        adata_e.var_names_make_unique()
        sc.pp.scrublet(adata_e)
        adata_e.write(PATH_scrublet_adata_e)    

/home/18parkky/anaconda3-2021.11/envs/sc/lib/python3.12/site-packages/anndata/_core/anndata.py:1820: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")
/home/18parkky/anaconda3-2021.11/envs/sc/lib/python3.12/site-packages/anndata/_core/anndata.py:1820: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")


### Merge h5ad

In [51]:
adata = list()
processed_sample_count = 0
unprocessed_sample_count = 0

for patientID, edf in chen_metadata.groupby('PatientID'):
    print(patientID, flush=True)
    for tup in edf.itertuples():
    
        if tup.BiopsySite == 'Normal':
            DIR_Chen_cellranger_out_temp = f'{DIR_Chen_cellranger_out}/Adjacent_normal_tissue'
        elif tup.BiopsySite == 'Tumor':
            DIR_Chen_cellranger_out_temp = f'{DIR_Chen_cellranger_out}/Tumor'
            
        PATH_cellranger_h5ad = f'{DIR_Chen_cellranger_out_temp}/{tup.Accession}/outs/filtered_feature_bc_matrix.h5'
        PATH_cellranger_scrublet_out = f'{DIR_Chen_cellranger_out_temp}/{tup.Accession}/outs/filtered_feature_bc_matrix.scrublet.h5'
        
        if os.path.exists( PATH_cellranger_h5ad ) == True:
            if os.path.exists( PATH_cellranger_scrublet_out ) == True:  # Add to list
                adata_temp = sc.read_h5ad( PATH_cellranger_scrublet_out )
                
                adata_temp.obs['SampleID']  = tup.Accession
                adata_temp.obs['PatientID'] = tup.PatientID
                adata_temp.obs['BiopsySite'] = tup.BiopsySite

                adata.append( adata_temp ) 
                pass
                
            else:   # Run scrublet, save to disk then add to list
                adata_temp = sc.read_10x_h5( PATH_cellranger_h5ad )
                adata_temp.obs_names_make_unique()
                adata_temp.var_names_make_unique()
                sc.pp.scrublet(adata_temp)
                adata_temp.write(PATH_cellranger_scrublet_out)
                adata.append( adata_temp )
                pass
        else:   # Report
            unprocessed_sample_count += 1
            print(f'{patientID}-{tup.Accession} has not been processed by CellRanger!')

P04
P08
P11
P12
P15
P16
P17
P18
P19
P20
P21
P22
P23
P24
P25
P26


In [52]:
adata = sc.concat( adata  )
adata.obs_names_make_unique()

# mitochondrial genes, "MT-" for human, "Mt-" for mouse
adata.var["mt"] = adata.var_names.str.startswith("MT-")
# ribosomal genes
adata.var["ribo"] = adata.var_names.str.startswith(("RPS", "RPL"))
# hemoglobin genes
adata.var["hb"] = adata.var_names.str.contains("^HB[^(P)]")

sc.pp.calculate_qc_metrics(
    adata, qc_vars=["mt", "ribo", "hb"], inplace=True, log1p=True
)

/home/18parkky/anaconda3-2021.11/envs/sc/lib/python3.12/site-packages/anndata/_core/anndata.py:1818: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")


In [55]:
adata.obs['Accession'] = adata.obs['SampleID']
adata.write('/node200data/18parkky/datasets/data/public/processed_data/Immunotherapy_CRC_Chen_et_al/1_raw_h5ad/Chen.MSI.raw.h5ad')